# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [9]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [10]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("C:/Users/jyots/ucb/hw/World_Weather_Analysis/Weather_Database.ipynb/Weather_Database_starter_code/Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Port Elizabeth,ZA,73.31,few clouds,-33.9180,25.5701,Waterford Hotel
1,3,Manokwari,ID,82.04,broken clouds,-0.8667,134.0833,Hotel Arfak
2,5,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,Paradise Inn
3,6,Kapaa,US,79.12,light rain,22.0752,-159.3190,Pono Kai Resort
4,7,Bougouni,ML,70.61,broken clouds,11.4170,-7.4790,Hotel Piemont
...,...,...,...,...,...,...,...,...
159,657,Dwarka,IN,80.35,clear sky,22.2394,68.9678,The Dwarika Hotel
160,665,Bako,ET,74.68,overcast clouds,5.7833,36.5667,Goh Hotel
161,679,Lodwar,KE,86.67,broken clouds,3.1191,35.5973,Lodwar Modern Lodge
162,680,Novo Aripuana,BR,72.32,scattered clouds,-5.1206,-60.3797,Hotel Novo Aripuanã


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [11]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [26]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City'] =='Narrabri']
vacation_end = vacation_df.loc[vacation_df['City'] =='Narrabri']
vacation_stop1 = vacation_df.loc[vacation_df['City'] =='Yeppoon']
vacation_stop2 = vacation_df.loc[vacation_df['City'] =='Charlestown']
vacation_stop3 = vacation_df.loc[vacation_df['City'] =='Brisbane']

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [27]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], axis=0)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
114,463,Narrabri,AU,81.34,moderate rain,-30.3167,149.7833,Tattersall's Hotel
98,401,Yeppoon,AU,73.18,overcast clouds,-23.1333,150.7333,Blossom's on Seaspray
90,373,Charlestown,AU,72.01,broken clouds,-32.9500,151.6667,Warners Bay Hotel
21,59,Brisbane,AU,71.83,light rain,-27.4679,153.0281,The Manor Apartment Hotel
114,463,Narrabri,AU,81.34,moderate rain,-30.3167,149.7833,Tattersall's Hotel


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [32]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["City", "Lng", "Lat"]].copy()

# Display sample data
waypoints_df

,City,Lng,Lat
114,Narrabri,149.7833,-30.3167
98,Yeppoon,150.7333,-23.1333
90,Charlestown,151.6667,-32.9500
21,Brisbane,153.0281,-27.4679
114,Narrabri,149.7833,-30.3167


## Use GeoViews to create map that shows the four cities in the itinerary

In [82]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = 0.001,
    scale = 500,
    color = "City"
)

In [83]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [70]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [74]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'f3c8090b0c9749649acac3ceac9de2e0',
 'waypoints': '-30.3167,149.7833|-23.1333,150.7333|-32.95,151.6667|-27.4679,153.0281|-30.3167,149.7833'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [78]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = requests.get(base_url, params=params).json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [149.7833, -30.3167], 'original_index': 0},
     {'location': [150.7333, -23.1333], 'original_index': 1},
     {'location': [151.6667, -32.95], 'original_index': 2},
     {'location': [153.0281, -27.4679], 'original_index': 3},
     {'location': [149.7833, -30.3167], 'original_index': 4}],
    'units': 'metric',
    'distance': 3599628,
    'distance_units': 'meters',
    'time': 133511.001,
    'legs': [{'distance': 933195,
      'time': 35067.265,
      'steps': [{'from_index': 0,
        'to_index': 6,
        'distance': 303,
        'time': 31.282,
        'instruction': {'text': 'Drive southwest on Namoi Street.'}},
       {'from_index': 6,
        'to_index': 439,
        'distance': 97141,
        'time': 3318.07,
        'instruction': {'text': 'Turn right onto A39/Barwan Street. Continue on A39.'}},
       {'from_index': 439,
        'to_index': 1534,
        'distance': 117051

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [87]:
# Fetch the route's legs coordinates from the JSON reponse
legs = fetch('https://api.geoapify.com/v1/routing/get/json')
   .then(response => response.json())
   .then(json => console.log(json))

IndentationError: unexpected indent (2866581118.py, line 3)

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [13]:
# Create and empty list to store the longitude of each step
# YOUR CODE HERE

# Create and empty list to store the latitude of step
# YOUR CODE HERE

# Loop through the legs coordinates to fetch the latitude and longitude for each step
# YOUR CODE HERE

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [14]:
# Create an empty DataFrame to store the steps' coordinates
route_df = # YOUR CODE HERE

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,longitude,latitude
0,12.479944,41.889982
1,12.479927,41.890012
2,12.479760,41.890249
3,12.479580,41.890493
4,12.479418,41.890662


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [15]:
# Configure the route path by using the GeoViews' Path function
route_path = # YOUR CODE HERE

In [16]:
# Display a composed plot by using the route_map and route_path objects
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]